# 本文件说明
- 需求时间：20180704
- 挑出不同机构的预警数据（title + content）分词计算词频
- 一条新闻涉及多家机构时会重复多次，算多条数据
- 按机构统计词频

# 基本设置

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import jieba
from collections import Counter
import pre_bank

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.948 seconds.
Prefix dict has been built succesfully.


# 导入数据

In [2]:
org_dict = {'263':'国家开发银行','265':'进出口银行','269':'邮政储蓄银行',
            '272':'工商银行','273':'农业银行','274':'中国银行','275':'建设银行',
            '276':'交通银行','278':'中信银行','279':'光大银行','280':'华夏银行',
            '281':'广华银行','282':'平安银行','283':'招商银行','284':'浦发银行',
            '285':'兴业银行','286':'民生银行','287':'恒丰银行','288':'浙商银行',
            '289':'渤海银行','1920':'农业发展银行'}

In [3]:
data = pd.read_excel('data/第二季度21家预警数据.xlsx')
data['title_content'] = data['title'] + data['content']
data['tree_node_id'] = data['tree_node_id'].apply(lambda x:org_dict[str(x)])
print(data.shape)
data.head()

(25683, 4)


,tree_node_id,title,content,title_content
0,国家开发银行,结构性存款激增9000亿!银行祭出揽储利器,证券时报记者 刘筱攸 强监管下，银行保本理财正逐渐萎缩。而替代它的产品之一，是结构...,结构性存款激增9000亿!银行祭出揽储利器 证券时报记者 刘筱攸 强监管下，银行保本...
1,国家开发银行,结构性存款激增9000亿!银行祭出揽储利器,强监管下，银行保本理财正逐渐萎缩。而替代它的产品之一，是结构性存款。 证券时报记者统计...,结构性存款激增9000亿!银行祭出揽储利器 强监管下，银行保本理财正逐渐萎缩。而替代它的产...
2,国家开发银行,结构性存款激增9000亿 银行祭出揽储利器,强监管下，银行保本理财正逐渐萎缩。而替代它的产品之一，是结构性存款。 证券时报记者统...,结构性存款激增9000亿 银行祭出揽储利器 强监管下，银行保本理财正逐渐萎缩。而替代它的产...
3,国家开发银行,结构性存款激增9000亿!银行祭出揽储利器,数据来源：金融机构信贷收支统计数据 东方IC/供图 郑加良/制表 证券时报记者 刘筱攸 ...,结构性存款激增9000亿!银行祭出揽储利器数据来源：金融机构信贷收支统计数据 东方IC/供图...
4,国家开发银行,结构性存款激增9000亿!银行祭出揽储利器,摘要 【结构性存款激增9000亿!银行祭出揽储利器】强监管下，银行保本...,结构性存款激增9000亿!银行祭出揽储利器摘要 【结构性存款激增900...


In [4]:
data['tree_node_id'].value_counts()

中国银行      5139
工商银行      3479
建设银行      3026
农业银行      2258
招商银行      1959
交通银行      1611
兴业银行      1264
中信银行      1175
民生银行       934
浦发银行       830
光大银行       656
浙商银行       651
广华银行       572
平安银行       563
华夏银行       395
邮政储蓄银行     312
国家开发银行     213
恒丰银行       211
进出口银行      205
农业发展银行     155
渤海银行        75
Name: tree_node_id, dtype: int64

# 导入字典

## 负面词

In [5]:
neg_1 = []
f = open('corpus/neg_words.txt', 'r+', encoding = 'utf-8')
for line in f.readlines():
    neg_1.append(line.strip().encode('utf-8').decode('utf-8-sig'))
f.close()    
print(len(neg_1))

1147


In [6]:
# neg_1[1080:]

In [7]:
neg_2 = []
f = open('corpus/negative_words.txt', 'r+', encoding = 'utf-8')
for line in f.readlines():
    neg_2.append(line.strip().encode('utf-8').decode('utf-8-sig'))
f.close() 
print(len(neg_2))

864


In [8]:
neg_3 = pd.read_excel('corpus/银行业微博负面词.xlsx')
print(neg_3.shape)
neg_3.head()

(243, 2)


,关键词,批次
0,傻叉,第一批
1,有毒,第一批
2,有待提高,第一批
3,垃圾,第一批
4,效率低,第一批


In [9]:
neg_4 = pd.read_excel('corpus/银行业负面词及预警词-长文本.xlsx')
print(neg_4.shape)
neg_4.head()

(604, 2)


,预警词,倾向性
0,ATM故障,NaN
1,SB,NaN
2,暗箱操作,NaN
3,罢工,NaN
4,霸道,NaN


In [10]:
neg_words = neg_1 + neg_2 + neg_3['关键词'].tolist() + neg_4['预警词'].tolist()
print('去重前：', len(neg_words))
neg_words = list(set(neg_words))
print('去重后：', len(neg_words))\

remove_list = ['银行',]
for remove_word in remove_list:
    if remove_word in  neg_words:
        neg_words.remove(remove_word)
        print('remove_word: ', remove_word)

去重前： 2858
去重后： 2589
remove_word:  银行


In [11]:
# x = '银行'
# if x in neg_words:
#     print(neg_words.index(x))
#     print(neg_words[neg_words.index(x)])

In [12]:
f = open('corpus/neg_words_20180704.txt', 'w+', encoding = 'utf-8')
for line in neg_words:
    f.write(line + '\n')
f.close()    

## 银行词典

In [13]:
bank_dict_1 = []
f = open('corpus/bank_dict.txt', 'r+', encoding = 'utf-8')
for line in f.readlines():
    bank_dict_1.append(line.strip().encode('utf-8').decode('utf-8-sig'))
f.close()    
print(len(bank_dict_1))

1354


In [14]:
bank_dict_2 = pd.read_excel('corpus/银行业词典v5.xlsx')
print(bank_dict_2.shape)
bank_dict_2.head()

(1560, 1)


,关键词
0,AMC
1,ATM
2,存取款机
3,ATM机
4,取款机


In [15]:
bank_dict_tmp = pd.read_excel('corpus/180511金融机构词表.xlsx')
print(bank_dict_tmp.shape)
bank_dict_tmp.tail()

(4432, 2)


,银行名称,银行同义词
4427,温岭市箬横镇玉麟农村资金互助社,玉麟农村资金互助社 玉麟资金互助社
4428,瑞安市马屿镇汇民农村资金互助社,汇民农村资金互助社 汇民资金互助社
4429,建德市大同镇桑盈农村资金互助社,桑盈农村资金互助社 桑盈资金互助社
4430,重庆市黔江区城东诚信农村资金互助社,黔江区城东诚信农村资金互助社 黔江城东诚信资金互助社 黔江诚信农村资金互助社 黔江诚信资金互助社
4431,重庆市江津区白沙镇明星农村资金互助社,明星农村资金互助社 明星资金互助社


In [16]:
bank_dict_3 = bank_dict_tmp['银行名称'].tolist()
bank_dict_4 = []
for words in bank_dict_tmp['银行同义词'].tolist():
    if str(words) != 'nan':
        word_list = str(words).split(' ')
        bank_dict_4 += word_list

In [17]:
bank_dict = bank_dict_1 + bank_dict_2['关键词'].tolist() + bank_dict_3 + bank_dict_4
print('去重前：', len(bank_dict))
bank_dict = list(set(bank_dict))
print('去重后：', len(bank_dict))

# remove_list = ['银行',]
# for remove_word in remove_list:
#     if remove_word in  neg_words:
#         neg_words.remove(remove_word)
#         print('remove_word: ', remove_word)

去重前： 23102
去重后： 20371


In [18]:
x = '银行'
if x in bank_dict_1:
    print(bank_dict_1.index(x))
    print(bank_dict_1[bank_dict_1.index(x)])

861
银行


In [19]:
# bank_dict_1[860:]

In [20]:
f = open('corpus/bank_dict_20180704.txt', 'w+', encoding = 'utf-8')
for line in bank_dict:
    if len(line) > 0:
        f.write(line + '\n')
f.close()   

## 预处理 + 分词 + 词频统计

In [21]:
class decide_neg():
    def __init__(self):
        self.neg_words = []
        f = open('corpus/neg_words_20180704.txt', 'r+', encoding = 'utf-8')
        for word in f.readlines():
            self.neg_words.append(word.replace('\n', ''))
        f.close()  
        
    def get_neg_label(self, x):
        for word in self.neg_words:
            if word == x:
#                 print('word', word)
#                 print('x', x)
                return 1
                break
        

In [22]:
for bank in data['tree_node_id'].unique():
    print('-- 处理机构：', bank)    
    bank_data = data[data['tree_node_id'] == bank]['title_content'].tolist()
    print('---- 共 %s 条数据'%len(bank_data)) 
    
    bank_pre = pre_bank.handle_contents(bank_data)
    
    word_count = {}
    for words in bank_pre:
        word_list = words.split(' ')
        for word in word_list:
            if len(word) <= 1: #排除单个字符的分词结果
                continue
            elif str(word) == 'nan':
                continue
            else :
                word_count[word]=word_count.get(word,0)+1
                
    word_data = pd.DataFrame.from_dict(word_count, orient='index').sort_values(by = 0, ascending = False).reset_index()
    word_data.columns = ['word', 'freq']
    y = decide_neg()
    word_data['neg'] = word_data['word'].apply(lambda x:y.get_neg_label(x))
    word_data.fillna(0).to_excel('data_count/%s_word_counts_20180704.xlsx'%bank, index = False)

-- 处理机构： 国家开发银行
---- 共 213 条数据
-- 处理机构： 进出口银行
---- 共 205 条数据
-- 处理机构： 邮政储蓄银行
---- 共 312 条数据
-- 处理机构： 工商银行
---- 共 3479 条数据
-- 处理机构： 农业银行
---- 共 2258 条数据
-- 处理机构： 中国银行
---- 共 5139 条数据
-- 处理机构： 建设银行
---- 共 3026 条数据
-- 处理机构： 交通银行
---- 共 1611 条数据
-- 处理机构： 中信银行
---- 共 1175 条数据
-- 处理机构： 光大银行
---- 共 656 条数据
-- 处理机构： 华夏银行
---- 共 395 条数据
-- 处理机构： 广华银行
---- 共 572 条数据
-- 处理机构： 平安银行
---- 共 563 条数据
-- 处理机构： 招商银行
---- 共 1959 条数据
-- 处理机构： 浦发银行
---- 共 830 条数据
-- 处理机构： 兴业银行
---- 共 1264 条数据
-- 处理机构： 民生银行
---- 共 934 条数据
-- 处理机构： 恒丰银行
---- 共 211 条数据
-- 处理机构： 浙商银行
---- 共 651 条数据
-- 处理机构： 渤海银行
---- 共 75 条数据
-- 处理机构： 农业发展银行
---- 共 155 条数据
